In [1]:
# %%
import pandas as pd
import re
# %%

### The above code just generates the cleaner_data.csv. So just work with cleaner_data.csv below to save 30min!

In [2]:
filename = 'cleaner_data.csv'
df = pd.DataFrame()
def auto_truncate(val):
    return val[:]
for tp in pd.read_csv(filename, chunksize = 25000, converters={'message': auto_truncate}):
    df = df.append(tp)
df

,file,body,Date,X-From,X-To,Subject,X-cc,X-bcc,Message-ID
0,allen-p/contacts/1.,\nloan servicing-jessica weeber 800-393-5626 j...,"Mon, 14 May 2001 17:18:42 -0700 (PDT)",Outlook-Migration-Team </O=ENRON/OU=NA/CN=RECI...,NaN,Lee Odonnel,NaN,NaN,<15816310.1075855374294.JavaMail.evans@thyme>
1,allen-p/contacts/2.,\nexit mccollough off 410,"Mon, 14 May 2001 17:18:42 -0700 (PDT)",Outlook-Migration-Team </O=ENRON/OU=NA/CN=RECI...,NaN,Greg Thorse,NaN,NaN,<6521706.1075855374316.JavaMail.evans@thyme>
2,allen-p/deleted_items/1.,"\n \n>From: ""Greg Thorse"" \n>To: \n>CC: ""Phill...","Sun, 30 Dec 2001 10:19:42 -0800 (PST)","""phillip allen"" <pallen70@hotmail.com>@ENRON",pallen@enron.com,Fwd: Bishops Corner,NaN,NaN,<21543395.1075855374340.JavaMail.evans@thyme>
3,allen-p/deleted_items/10.,\nThis request has been pending your approval ...,"Thu, 27 Dec 2001 17:16:46 -0800 (PST)",ARSystem <ARSystem@mailman.enron.com>@ENRON,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...",Your Approval is Overdue: Access Request for m...,NaN,NaN,<25363451.1075855374674.JavaMail.evans@thyme>
4,allen-p/deleted_items/100.,"\nIf you cannot read this email, please click ...","Thu, 18 Oct 2001 15:21:22 -0700 (PDT)","""Earnings.com"" <webmaster@earnings.com>@ENRON",pallen@enron.com,NT Earnings Information,NaN,NaN,<19705494.1075858631723.JavaMail.evans@thyme>
...,...,...,...,...,...,...,...,...,...
254278,zufferli-j/sent_items/95.,\nThis is a trade with OIL-SPEC-HEDGE-NG (John...,"Wed, 28 Nov 2001 13:30:11 -0800 (PST)","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Loibl, Kori </O=ENRON/OU=NA/CN=RECIPIENTS/CN=K...",Trade with John Lavorato,NaN,NaN,<26807948.1075842029936.JavaMail.evans@thyme>
254279,zufferli-j/sent_items/96.,\nSome of my position is with the Alberta Term...,"Wed, 28 Nov 2001 12:47:48 -0800 (PST)","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",Gas Hedges,NaN,NaN,<25835861.1075842029959.JavaMail.evans@thyme>
254280,zufferli-j/sent_items/97.,\n2\n -----Original Message-----\nFrom: \tDouc...,"Wed, 28 Nov 2001 07:20:00 -0800 (PST)","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Doucet, Dawn </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",RE: CONFIDENTIAL,NaN,NaN,<28979867.1075842029988.JavaMail.evans@thyme>
254281,zufferli-j/sent_items/98.,\nAnalyst\t\t\t\t\tRank\n\nStephane Brodeur\t\...,"Tue, 27 Nov 2001 11:52:45 -0800 (PST)","Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",Calgary Analyst/Associate,NaN,NaN,<22052556.1075842030013.JavaMail.evans@thyme>


In [3]:
def file_to_email():
    valid = set()
    valid.add('no.address@enron.com')
    for file in df['file']:
        i = file.find('/')
        #if statements are for spelling inconsistentcies in the csv
        if file[:i] == 'crandell-s':
            valid.add('crandall-s@enron.com')
        elif file[:i] == 'merriss-s':
            valid.add('merris-s@enron.com')
        elif file[:i] == 'phanis-s':
            valid.add('panus-s@enron.com')
        elif file[:i] == 'rodrique-r':
            valid.add('rodrigue-r@enron.com')
        else:
            valid.add(file[:i]+'@enron.com')
    return list(valid)

In [4]:
# Surname, Name
def lastcommafirst(name):
    i = name.find(',')
    j = name.find(' ', i+2)
    if j == -1: j = len(name)
    sname = name[:i].capitalize()
    l = sname.find(' ')
    if l >= 0: sname=sname[:l]
    fname = name[i+2:j].capitalize()
    sname = re.sub('[\W_]', '', sname)
    if len(fname) > 0 and len(sname) > 0:
        return [fname, sname]
    else:
        return []

# Name (middle )Surname
def firstmidlast(name):
    i = name.find(' ')
    j = name.rfind(' ')
    fname = name[:i].capitalize()
    sname = name[j+1:].capitalize()
    sname = re.sub('[\W_]', '', sname)
    if len(fname) > 0 and len(sname) > 0:
        return [fname, sname]
    else:
        return []


def string_to_name(name):
    k = name.find('<')
    if name == 'Patrice L Mims':
        return ['Patrice', 'Mims-Thurston']
    # Has <>
    if k >= 0:
        name = name[:k-1]
        #"Name Surname"
        if name[0] == '\"':
            i = name.find('\"',1)
            name = name[1:i]
            # No space (invalid)
            if name.find(' ') == -1:
                return []
            # Has space
            else:
                return firstmidlast(name)                

        #No quotation marks
        else:
            # string-with-no-space (is never valid)
            if name.find(' ') == -1:
                return []
            # Surname, Name
            else:
                if name[:18] == 'Williams III, Bill':
                    return ['Bill', 'Williams', 'williams-w3@enron.com']
                return lastcommafirst(name)
            
    # No <>            
    else:
       #Surname, Name
        if ',' in name:
            return lastcommafirst(name)
        #Name Surname@
        elif '@' in name:
            i = name.find('@')
            name = name[:i]
            return firstmidlast(name)
        #Name( middle) Surname
        else:
            if name == 'Carol St Clair':
                return ['Carol', 'StClair']
            if name == 'Liz M Taylor':
                return ['Liz', 'Taylor', 'whalley-l@enron.com']
            return firstmidlast(name)
            

    

In [5]:
#%%ignore function, might help for To?
def valid_name(mail_name):
    if len(mail_name) >= 2:
        mail = (mail_name[1]+'-'+mail_name[0][0]+'@enron.com').lower()
        if len(mail_name) == 3:
            mail = mail_name[2]
        if re.match(email_reg, mail) and mail in valid_email:
            return mail
        else:
            return 'no.address@enron.com'
    else:
        return 'no.address@enron.com'


In [6]:
email_reg = r"^[a-z]+\-[\da-z-]+@enron.com$"

valid_email = file_to_email()
employ = []
mess = []
ref =[]
no_ad = [None, None, 'no.address@enron.com']
employ.append(no_ad)


for x in range(len(df)):
#for x in range(0, 1000):  #if you just wanna test on a small bit of the table first

    row = df.iloc[x].tolist()
    from_name = string_to_name(row[3])
    if len(from_name) >= 2:
        from_mail = (from_name[1]+'-'+from_name[0][0]+'@enron.com').lower()
        if len(from_name) == 3:
            from_mail = from_name[2]
        if re.match(email_reg, from_mail) and from_mail in valid_email:
            from_name.append(from_mail)
            employ.append(from_name[:3])
        else:
            from_mail = 'no.address@enron.com'
    else:
        from_mail = 'no.address@enron.com'
    row[3] = from_mail
     
#%% below is my attempt at seperating emails in To field (doesn't work, YET)
#    xto = str(row[4])  
#    if xto == 'nan':
#        row[4] = 'undisclosed-recepients@enron.com'
#        all_to_name = 'undisclosed-recepients@enron.com'
#    else:
#        xto = xto.replace('\n', '')
#        print(xto, '\t\t\t', x)
#        all_to_name = set()
#        m = xto.count(',')
#        for y in range(m+1):
#            if y < m :
#                i = xto.find('>,')
#                to_name = string_to_name(xto[:i+1])
#                xto = xto[i+1:]
#            else:
#                to_name = string_to_name(xto)
#            #print(to_name)
#            all_to_name.add(valid_name(to_name))    
#        #print(all_to_name)
#        row[4] = list(all_to_name)
    #df.loc[x] = row

employdf = pd.DataFrame(employ)
employdf = employdf.drop_duplicates(2, keep = 'first') #removes dup emails
employdf = employdf.sort_values(2)
pd.DataFrame.to_csv(employdf, 'EmployeeList.csv')